## UI Code with visualization
### (Run project_final_code before running this ui code)

In [8]:
import os
import json
import pandas as pd
import dash
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from PyPDF2 import PdfReader
from docx import Document
import plotly.express as px

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

# Store results for easy access in callbacks
results_store = []

# Function to read files
def read_file_with_fallback(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    elif file_path.endswith('.pdf'):
        reader = PdfReader(file_path)
        return " ".join(page.extract_text() for page in reader.pages)
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        return " ".join(paragraph.text for paragraph in doc.paragraphs)
    else:
        raise ValueError("Unsupported file format")

# Define the layout with navigation
app.layout = dbc.Container([
    dcc.Location(id='url', refresh=False),
    dbc.NavbarSimple(
        children=[
            dbc.NavItem(dbc.NavLink("Home Page", href='/')),
            dbc.NavItem(dbc.NavLink("Results", href='/results')),
            dbc.NavItem(dbc.NavLink("Visualization", href='/visualization'))  # Added Visualization link
        ],
        brand=html.Div("Kyndryl", style={'color': '#FF0000', 'fontSize': '24px', 'fontWeight': 'bold', 'backgroundColor': '#000000', 'padding': '10px'}),
        brand_href="/",
        color="black",
        dark=True,
    ),
    html.Div(id='page-content')
], fluid=True)

# Define the CV scoring page layout
cv_scoring_layout = dbc.Container([
    dbc.Row([ 
        dbc.Col(html.H1("Smart ATS", className="text-center", 
                         style={'color': '#FFFFFF', 'backgroundColor': '#FF0000', 'padding': '10px', 'borderRadius': '10px'}),
                className="mb-4")
    ]),
    dbc.Row([ 
        dbc.Col(html.Label("Select Job Description:", style={'color': '#FF0000'}),
                className="mb-2"),
        dbc.Col(dcc.Dropdown(id='jd-dropdown', options=[ 
            {'label': filename, 'value': filename} for filename in os.listdir('Dataset/job_descriptions/') if filename.endswith(('.txt', '.pdf', '.docx'))
        ], 
        placeholder="Select a Job Description",
        style={'backgroundColor': '#F5F5F5', 'borderRadius': '5px'}), className="mb-4",
        style={'backgroundColor': '#000000', 'padding': '10px', 'borderRadius': '10px'})
    ]),
    dbc.Row([ 
        dbc.Col(dbc.Button('Submit', id='submit-button', color='light', n_clicks=0, 
                           style={'backgroundColor': '#FF0000', 'border': 'none', 'color': '#FFFFFF', 'fontWeight': 'bold'}), 
                className="mb-4", width={'size': 2, 'offset': 5})
    ]),
    dbc.Row([ 
        dbc.Col(html.Div(id='output-container', 
                         style={'backgroundColor': '#FFFFFF', 'padding': '20px', 'borderRadius': '10px', 'border': '2px solid #FF0000'}), 
                width=12)
    ])
], fluid=True, style={'backgroundColor': '#F5F5F5', 'min-height': '100vh', 'padding': '20px'})

# Define the results page layout
results_layout = dbc.Container([
    dbc.Row([ 
        dbc.Col(html.H1("Candidate Ranking", className="text-center", 
                         style={'color': '#FFFFFF', 'backgroundColor': '#FF0000', 'padding': '10px', 'borderRadius': '10px'}),
                className="mb-4")
    ]),
    dbc.Row([ 
        dbc.Col(html.Div(id='results-table', style={'padding': '20px'}), width=12)
    ])
], fluid=True, style={'backgroundColor': '#F5F5F5', 'min-height': '100vh', 'padding': '20px'})

# Define the visualization page layout
visualization_layout = dbc.Container([
    dbc.Row([ 
        dbc.Col(html.H1("Results Visualization", className="text-center", 
                         style={'color': '#FFFFFF', 'backgroundColor': '#FF0000', 'padding': '10px', 'borderRadius': '10px'}),
                className="mb-4")
    ]),
    # Add the graph which will be hidden initially
    dbc.Row([ 
        dbc.Col(dcc.Graph(id='results-graph', style={'height': '60vh'}), width=12)  # Initially hidden
    ])
], fluid=True, style={'backgroundColor': '#F5F5F5', 'min-height': '100vh', 'padding': '20px'})

# Process job description and CVs, then compute scores
def update_output(n_clicks, selected_jd_filename):
    if n_clicks > 0 and selected_jd_filename:
        jd_path = 'Dataset/job_descriptions/'
        cv_path = 'Dataset/resumes/'
        ref_data_path = 'ReferenceData/'
        
        skill_files = [os.path.join(ref_data_path, f) for f in ['library.txt', 'global_skill.txt', 'skillset.txt']]
        inst_files = [os.path.join(ref_data_path, f) for f in ['INST.txt', 'tier1.txt', 'tier2.txt']]
        mnc_file = os.path.join(ref_data_path, 'mnc.txt')

        jd_text = read_file_with_fallback(os.path.join(jd_path, selected_jd_filename))
        jd_skills = extract_priority_skills(jd_text, skill_files)

        results = []
        for cv_filename in os.listdir(cv_path):
            if cv_filename.endswith(('.txt', '.pdf', '.docx')):
                full_cv_path = os.path.join(cv_path, cv_filename)
                try:
                    cv_text = read_file_with_fallback(full_cv_path)
                    similarity_score = compute_similarity(jd_text, cv_text)

                    matched_skills = extract_cv_details(cv_text, jd_skills)
                    mnc_match, institution_match = filter_based_on_mnc_and_institution(cv_text, mnc_file, inst_files)
                    years_of_experience = extract_years_of_experience(cv_text)

                    skill_match_score = len(matched_skills) * 10
                    rfm_score = compute_rfm_score(matched_skills, mnc_match, institution_match, years_of_experience)
                    similarity_percentage = round(similarity_score * 100, 2)
                    weighted_score = compute_weighted_score(similarity_score, skill_match_score, rfm_score)
                    weighted_percentage = round(weighted_score * 100, 2)

                    results.append({
                        'CV Filename': cv_filename,
                        'Weighted Score': f"{weighted_percentage}%",
                        'Details': {
                            'Similarity Score': f"{similarity_percentage}%",
                            'Skill Match Score': skill_match_score,
                            'RFM Score': rfm_score,
                            'Years of Experience': years_of_experience,
                            'Matched Skills': ", ".join(matched_skills),
                            'MNC Match': mnc_match,
                            'Institution Match': institution_match
                        },
                        'Details Visible': False  # Flag to manage details visibility
                    })
                except Exception as e:
                    print(f"Failed to process {cv_filename}: {e}")

        # Store results for access in more details callback
        global results_store
        results_store = sorted(results, key=lambda x: (x['Weighted Score']), reverse=True)
        return "Results processed successfully.", '/results'

    return "Select a Job Description and press Submit.", '/'

@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname')
)
def display_page(pathname):
    if pathname == '/results':
        return results_layout
    elif pathname == '/visualization':
        return visualization_layout  # Navigate to the visualization layout
    else:
        return cv_scoring_layout

@app.callback(
    Output('output-container', 'children'),
    Output('url', 'pathname'),  # Redirect to results page after submission
    Input('submit-button', 'n_clicks'),
    State('jd-dropdown', 'value')
)
def handle_submit(n_clicks, selected_jd_filename):
    return update_output(n_clicks, selected_jd_filename)

@app.callback(
    Output('results-table', 'children'),
    Input('url', 'pathname'),
    Input({'type': 'details-button', 'index': dash.dependencies.ALL}, 'n_clicks')
)
def display_results(pathname, n_clicks):
    if pathname == '/results' and results_store:
        results_table = []
        
        for rank, result in enumerate(results_store, start=1):  # Start rank at 1
            # Create button to show more details
            details_button = dbc.Button("More Details", id={'type': 'details-button', 'index': rank}, color='dark', n_clicks=0, 
                                         style={'backgroundColor': '#FFA500', 'border': '2px solid #ffcc00', 'color': 'white', 'fontWeight': 'bold'})  # Button in orange

            # Check if the details button for this rank was clicked
            if n_clicks and n_clicks[rank - 1]:  # Adjust for rank-based indexing
                result['Details Visible'] = not result['Details Visible']  # Toggle visibility

             # Define alternating row colors
            row_style = {'backgroundColor': '#FFFFFF'} if rank % 2 == 0 else {'backgroundColor': '#FF0000', 'color': '#FFFFFF'}
            
            result_row = html.Tr([
                html.Td(rank, style={'fontWeight': 'bold', 'border': '2px solid black'}),  # Display rank
                html.Td(result['CV Filename'], style={'border': '2px solid black'}),
                html.Td(result['Weighted Score'], style={'border': '2px solid #FF0000', 'fontWeight': 'bold'}),
                html.Td(details_button, style={'border': '2px solid black'})
            ], style=row_style)
            results_table.append(result_row)

            # Display more details if 'Details Visible' is True
            if result['Details Visible']:
                details_row = html.Tr([
                    html.Td(colSpan=4, children=[
                        html.Ul([
                            html.Li(f"Similarity Score: {result['Details']['Similarity Score']}", style={'backgroundColor': '#F5F5DC', 'color': '#FF0000'}),
                            html.Li(f"Skill Match Score: {result['Details']['Skill Match Score']}", style={'backgroundColor': '#F5F5DC', 'color': '#FF0000'}),
                            html.Li(f"RFM Score: {result['Details']['RFM Score']}", style={'backgroundColor': '#F5F5DC', 'color': '#FF0000'}),
                            html.Li(f"Years of Experience: {result['Details']['Years of Experience']}", style={'backgroundColor': '#F5F5DC', 'color': '#FF0000'}),
                            html.Li(f"Matched Skills: {result['Details']['Matched Skills']}", style={'backgroundColor': '#F5F5DC', 'color': '#FF0000'}),
                            html.Li(f"MNC Match: {result['Details']['MNC Match']}", style={'backgroundColor': '#F5F5DC', 'color': '#FF0000'}),
                            html.Li(f"Institution Match: {result['Details']['Institution Match']}", style={'backgroundColor': '#F5F5DC', 'color': '#FF0000'})
                        ])
                    ])
                ])
                results_table.append(details_row)

        return dbc.Table([
            html.Thead(html.Tr([
                html.Th("Rank", style={'fontWeight': 'bold', 'border': '2px solid black'}),
                html.Th("CV Filename", style={'fontWeight': 'bold', 'border': '2px solid black'}),
                html.Th("Weighted Score", style={'fontWeight': 'bold', 'border': '2px solid black'}),
                html.Th("Details", style={'fontWeight': 'bold', 'border': '2px solid black'})
            ])),
            html.Tbody(results_table)
        ], bordered=True, hover=True, responsive=True, style={'border': '2px solid black'})

    return "No results yet."

# Callback to update the visualization
@app.callback(
    Output('results-graph', 'figure'),
    Input('url', 'pathname')
)
def update_graph(pathname):
    if pathname == '/visualization' and results_store:
        df = pd.DataFrame(results_store)
        fig = px.bar(df, x='CV Filename', y='Weighted Score',
                     title='CV Weighted Scores',
                     labels={'CV Filename': 'CV Names', 'Weighted Score': 'Weighted Score'},
                     color='Weighted Score',
                     color_continuous_scale=px.colors.sequential.Viridis)
        fig.update_layout(xaxis_title='CV Names', yaxis_title='Weighted Scores', xaxis_tickangle=-45)
        return fig
    return px.bar(title="No Data Available")


if __name__ == '__main__':
    app.run_server(debug=True)
